# Project description
Urban mobility significantly contributes to global environmental impact. With the growth in urbanareas worldwide, cities are implementing policies to focus on sustainable urban and mobility devel-opment. Urban areas exhibit high variability in structure, economic output, mobility behaviour,among others. Pinpointing to the exact causes of emissions is impossible. However, understandingthe relationship of mobility with infrastructure resources and other urban indicators may help gov-ernments predict long-term mobility behaviour and intervene with future pathways into sustainablemobility options. </br>

Project goals: You are tasked to identify how mobility is related to the infrastructural form of acity and across multiple cities. Use your findings toward helping governments transform mobilitywithin cities for reducing environmental impacts of mobility use.
Data: </br>
1. Local city-wide indicators of urban mobility: Multi-city Traffic Datasetprovided publicly byThe Institute for Transport Planning and Systems at ETH Zurich.
2. Global city-wide indicators of urban mobility and environmental impacts:Urban TypologiesProject[Reference]3.Get indicators for infrastructure resources from OSMNX (street networks) and PYROSM(amenities and points of interest) python packages.High-level project goals1.Explore, Investigate, and Visualize various factors of the mobility data across cities of interest.2.Describe and incorporate additional data sources that you will use to help you understandrelationships between mobility and the built environment.
3. Identify correlations between mobility and the built environment.4.Perform clustering or LISA analysis on city streets where congestion may be related stronglywith the presence of certain amenities. Think about how that may vary with distance toamenities.
4. Or, train and evaluate models using the mobility data (either as predictors or as the primaryresponse in some fashion) and your data sources.
6. Use and interpret your models to discuss the causes and correlations of or effects due todifferences in congestion across cities or within a city of your choice.
7. Aggregate city-wide findings to correlate with environmental indicators used in data set (2) -urban typology - to comment on policy initiatives of the governments. (Think how you mayaggregate city-wide findings to a single score for the whole city to compare with indicatorsused in data set 

(2))Note: Not all data may be available for all cities. Hence, conduct this analysis for at least5citiesoverlapping between sets1and2in the data. You are welcome to do the analysis in many morecities.


### UTD-19

### OSMNX

In [1]:
# import 
import pandas as pd
import geopandas as gpd
import osmnx as ox
import matplotlib.pyplot as plt
import numpy as np

In [2]:
UTD19 = pd.read_csv('Data/detectors_public.csv') # read big file with all cities 
newcitylist= [ 'munich', 'rotterdam', 'frankfurt', 'hamburg', 'zurich'] # find only our cities for now, store in dataframe
UTD19Cities = UTD19.loc[UTD19.citycode.isin(newcitylist)] # UTD19Cities is dataframe

FileNotFoundError: [Errno 2] No such file or directory: 'Data/detectors_public.csv'

### OSMNX Graph creation

In [3]:
def GetGraphs(clist, df):
    # function designed to extract bbox and create graphs with bbox
    GraphList = []
    for i in clist:
        maxlonglt = df.loc[df.citycode == i, ['long', 'lat']].max() # maxlonglat[0] = long, 1 lat
        minlonglt = df.loc[df.citycode == i, ['long', 'lat']].min() #
        north = maxlonglt[1] 
        south = minlonglt[1]
        east = maxlonglt[0]
        west = minlonglt[0]
        A = ox.graph.graph_from_bbox(north, south, east, west, network_type='drive')
        GraphList.append(A)
    return GraphList
        
Graphs = GetGraphs(newcitylist, UTD19Cities)

NameError: name 'newcitylist' is not defined

In [ ]:
# example of plotting a graph
G1 = ox.project_graph(Graphs[1])
fig, ax = ox.plot_graph(G1, node_size=1)

In [ ]:
# Edge attributes and node data
Rotterdam = Graphs[1]
node_data, edge_attributes = ox.graph_to_gdfs(Rotterdam) # Get node info and edge info

NameLink = pd.DataFrame(edge_attributes[['osmid','name']]) # only extract the osmid and road name
NameLink.rename({'name': 'road'}, inplace = True, axis = 1) # The Graph network has more raods in it than UTD
# some entries are a list of street names 
NameLink = NameLink.explode('road') # use this to create entry with same values, so road is not a list anymore
#NameLink = NameLink.explode('osmid') # some roads have two osmid -> explode

URotterdam = UTD19Cities.loc[UTD19Cities.citycode == 'rotterdam', ['detid', 'road']] #only get detid and road name

TranslateTable = pd.merge(NameLink, URotterdam, on = 'road', how = 'right')
TranslateTable.head() # links osmid and detid

In [ ]:
# way to add colors to edges
x = [1 if x%2 == 0 else 0 for x in edge_attributes.index] # create some random atribute

edge_attributes['X']=x # set att

WithX = ox.graph_from_gdfs(node_data, edge_attributes, graph_attrs={'edge_attr':'X'}) # graph_attrs and edge works it is safed! and able to call back
WithX.graph['crs'] = Rotterdam.graph['crs'] # need crs to plot
ec = ox.plot.get_edge_colors_by_attr(WithX, attr='length', cmap='viridis') # set color scale by atribute (now length could be any other variable)

G1 = ox.project_graph(WithX)
fig, ax = ox.plot_graph(G1, node_size=1, edge_color=ec) # edgecolor is stored in ec

# Urban typologies import


In [ ]:
citylist = ['Munich', 'Rotterdam-Hague', 'Frankfurt', 'Hamburg', 'Zurich']
UrbanTypo = pd.read_excel('Data/FINAL-COMBINED-DATASET.xlsx')
Cities = UrbanTypo.loc[(UrbanTypo.City.isin(citylist))]
Cities.reset_index(inplace = True)
Cities.head()

# Fabians Merge

In [ ]:
Rotterdam = Graphs[1]
node_data, edge_attributes = ox.graph_to_gdfs(Rotterdam) # Get node info and edge info
#switch first and second column
colnames=[]
for i, names in enumerate(edge_attributes.columns):
    colnames.append(names)
colnames[0]='name'
colnames[1]='osmid'
#print(colnames)
edge_attributes=edge_attributes.reindex(columns=colnames)
edge_attributes.head()

In [ ]:
dfea=edge_attributes#chiller, want korter
dfea.loc[:,['name']]=np.nan_to_num(dfea.loc[:,['name']])

#create list to log whether a row contains a list for osmid
listornot=[]
for i in range(len(dfea.name)):
    if type(dfea.iloc[i,0]) == list:
        listornot.append(True)
    else:
        listornot.append(False)
#now append to df!
dfea['list?']=listornot
dfea.head(2)

In [ ]:
#save lists 
dfea_lists=dfea.loc[(dfea['list?'] == True)]
#save nonlists
dfea_nonlists=dfea.loc[(dfea['list?'] == False)]
dfea_lists=dfea_lists.reset_index()
#dfea_lists.drop(['index'])
dfea_lists=dfea_lists.iloc[:,1:17]
dfea_lists.head()

In [ ]:
#thrid try, also put appends in loop.
cdict = {name: [] for name in (dfea_lists.columns)} #make dict that consists out of lists equal to columns of df to fill below, with right names

for i in range (len(dfea_lists.name)):
    for j in range(len(dfea_lists.name[i])):
        ab=dfea_lists.name[i]
        cdict['name'].append(ab[j])
        for k, names_to_append in enumerate(dfea_lists.columns[1:len(dfea_lists.columns)]):
            l=k+1
            cdict[names_to_append].append(dfea_lists.iloc[i,l])
        
dfnew=pd.DataFrame.from_dict(cdict)
dfnew.head()
#works! 

In [ ]:
frames=[dfea_nonlists,dfnew]
final_set=pd.concat(frames)
final_set.head() # this is the final mergable set

### PYROSM Points of Interest

In [ ]:
# de filepaths zin nu specifiek voor mijn computer, moet je zelf even aanpassen naar waar die van jou staan
fp_rotterdam ="Documents/EPA/Urban Data Science/Final assignment/data/rotterdam.osm.pbf"
fp_munich ="C:/Users/hanna/Documents/EPA/Urban Data Science/Final assignment/data/munich.osm.pbf"
fp_zurich ="C:/Users/hanna/Documents/EPA/Urban Data Science/Final assignment/data/zurich.osm.pbf"
fp_hamburg ="C:/Users/hanna/Documents/EPA/Urban Data Science/Final assignment/data/hamburg.osm.pbf"
fp_frankfurt ="C:/Users/hanna/Documents/EPA/Urban Data Science/Final assignment/data/frankfurt.osm.pbf"

In [ ]:
for i in [fp_rotterdam, fp_munich, fp_zurich, fp_hamburg, fp_frankfurt]:
    # Initiliaze with bounding box
    osm = OSM(filepath=i)

    # Read POIs with custom filter 
    my_filter={'building': True} #"amenity": True, 'shop': True, }
    pois = osm.get_pois(custom_filter=my_filter)

    # Merge poi type information into a single column
    #pois["shop"] = pois["shop"].fillna(' ')
    #pois["amenity"] = pois["amenity"].fillna(' ')
    pois["building"] = pois["building"].fillna(' ')
    pois["poi_type"] = pois["building"] #+ pois["amenity"] + pois["shop"] 

    # Plot
    ax = pois.plot(column="poi_type", legend=True, markersize=4, figsize=(14,8), legend_kwds=dict(loc='upper left', ncol=2, bbox_to_anchor=(1, 1)))

# Mean flow per Detid

In [ ]:
# Read in csv-file per City
data_frankfurt = pd.read_csv('data\\UTD_f.csv')
data_hamburg = pd.read_csv('data\\UTD_h.csv')
data_munich = pd.read_csv('data\\UTD_m.csv')
data_rotterdam = pd.read_csv('data\\UTD_r.csv')
data_zurich = pd.read_csv('data\\UTD_z.csv')

Create tables with average per detid.

In [ ]:
frankfurt_detid_average = (data_frankfurt.groupby('detid').flow.mean()).to_frame(name = None)
hamburg_detid_average = (data_hamburg.groupby('detid').flow.mean()).to_frame(name = None)
munich_detid_average = (data_munich.groupby('detid').flow.mean()).to_frame(name = None)
rotterdam_detid_average = (data_rotterdam.groupby('detid').flow.mean()).to_frame(name = None)
zurich_detid_average = (data_zurich.groupby('detid').flow.mean()).to_frame(name = None)

Add the name of the city to each table.

In [ ]:
frankfurt_detid_average['city'] = 'frankfurt'
hamburg_detid_average['city'] = 'hamburg'
munich_detid_average['city'] = 'munich'
rotterdam_detid_average['city'] =  'rotterdam'
zurich_detid_average['city'] = 'zurich'

In [ ]:
#Show example of table
zurich_detid_average.head()

Concatenate the different tables.

In [ ]:
Mean_perdetid = pd.concat([frankfurt_detid_average,
                                  hamburg_detid_average,
                                  munich_detid_average,
                                  rotterdam_detid_average,
                                  zurich_detid_average])

In [ ]:
#show example of table
Mean_perdetid.head()

In [ ]:
#Check if number of unique detid is equal to total length of the table, this indicates that there are no overlapping detids
if  (len(Mean_perdetid) == Mean_perdetid.index.nunique()):
    print('all detids are unique')

In [ ]:
#Save Result
result_unique  = Mean_perdetid.to_csv('data\\UTD_Mean_Flow_per_detid.csv', index = True)#exporting data frame as csv file.